In [1]:
import sys
import os.path as osp
import gym.spaces
import itertools
import numpy as np
import random
import tensorflow                as tf
import tensorflow.contrib.layers as layers
from collections import namedtuple
from dqn_utils import *
from gym import wrappers
from atari_wrappers import *


OptimizerSpec = namedtuple("OptimizerSpec", ["constructor", "kwargs", "lr_schedule"])

In [2]:
def get_session():
    tf.reset_default_graph()
    tf_config = tf.ConfigProto(
        inter_op_parallelism_threads=1,
        intra_op_parallelism_threads=1)
    session = tf.Session(config=tf_config)
    return session

In [3]:
def atari_model(img_in, num_actions, scope, reuse=False):
    # as described in https://storage.googleapis.com/deepmind-data/assets/papers/DeepMindNature14236Paper.pdf
    with tf.variable_scope(scope, reuse=reuse):
        out = img_in
        with tf.variable_scope("convnet"):
            # original architecture
            out = layers.convolution2d(out, num_outputs=32, kernel_size=8, stride=4, activation_fn=tf.nn.relu)
            out = layers.convolution2d(out, num_outputs=64, kernel_size=4, stride=2, activation_fn=tf.nn.relu)
            out = layers.convolution2d(out, num_outputs=64, kernel_size=3, stride=1, activation_fn=tf.nn.relu)
        out = layers.flatten(out)
        with tf.variable_scope("action_value"):
            out = layers.fully_connected(out, num_outputs=512,         activation_fn=tf.nn.relu)
            out = layers.fully_connected(out, num_outputs=num_actions, activation_fn=None)

        return out

In [4]:
num_timesteps = 1000000
num_iterations = float(num_timesteps) / 4.0

lr_multiplier = 1.0
lr_schedule = PiecewiseSchedule([
                                     (0,                   1e-4 * lr_multiplier),
                                     (num_iterations / 10, 1e-4 * lr_multiplier),
                                     (num_iterations / 2,  5e-5 * lr_multiplier),
                                ],
                                outside_value=5e-5 * lr_multiplier)
optimizer = OptimizerSpec(
    constructor=tf.train.AdamOptimizer,
    kwargs=dict(epsilon=1e-4),
    lr_schedule=lr_schedule
)

In [5]:
benchmark = gym.benchmark_spec('Atari40M')

# Change the index to select a different game.
task = benchmark.tasks[3]

# Run training
seed = 0 # Use a seed of zero (you may want to randomize the seed!)
env = gym.make(task.env_id)
# set_global_seeds(seed)
# env.seed(seed)

expt_dir = '/tmp/hw3_vid_dir2/'
env = wrappers.Monitor(env, osp.join(expt_dir, "gym"), force=True)
env = wrap_deepmind(env)



session = get_session()
# tf.global_variables_initializer().run()

[2018-06-21 01:29:16,983] Making new env: PongNoFrameskip-v4
[2018-06-21 01:29:17,118] Clearing 4 monitor files from previous run (because force=True was provided)


In [6]:
env = env
q_func = atari_model
optimizer_spec = optimizer
session =  session
exploration=LinearSchedule(1000000, 0.1)
stopping_criterion=None
replay_buffer_size=1000000
batch_size=32
gamma=0.99
learning_starts=50000
learning_freq=4
frame_history_len=4
target_update_freq=10000
grad_norm_clipping=10
"""Run Deep Q-learning algorithm.

You can specify your own convnet using q_func.

All schedules are w.r.t. total number of steps taken in the environment.

Parameters
----------
env: gym.Env
    gym environment to train on.
q_func: function
    Model to use for computing the q function. It should accept the
    following named arguments:
        img_in: tf.Tensor
            tensorflow tensor representing the input image
        num_actions: int
            number of actions
        scope: str
            scope in which all the model related variables
            should be created
        reuse: bool
            whether previously created variables should be reused.
optimizer_spec: OptimizerSpec
    Specifying the constructor and kwargs, as well as learning rate schedule
    for the optimizer
session: tf.Session
    tensorflow session to use.
exploration: rl_algs.deepq.utils.schedules.Schedule
    schedule for probability of chosing random action.
stopping_criterion: (env, t) -> bool
    should return true when it's ok for the RL algorithm to stop.
    takes in env and the number of steps executed so far.
replay_buffer_size: int
    How many memories to store in the replay buffer.
batch_size: int
    How many transitions to sample each time experience is replayed.
gamma: float
    Discount Factor
learning_starts: int
    After how many environment steps to start replaying experiences
learning_freq: int
    How many steps of environment to take between every experience replay
frame_history_len: int
    How many past frames to include as input to the model.
target_update_freq: int
    How many experience replay rounds (not steps!) to perform between
    each update to the target Q network
grad_norm_clipping: float or None
    If not None gradients' norms are clipped to this value.
"""
assert type(env.observation_space) == gym.spaces.Box
assert type(env.action_space)      == gym.spaces.Discrete

In [7]:
###############
# BUILD MODEL #
###############

if len(env.observation_space.shape) == 1:
    # This means we are running on low-dimensional observations (e.g. RAM)
    input_shape = env.observation_space.shape
else:
    img_h, img_w, img_c = env.observation_space.shape
    input_shape = (img_h, img_w, frame_history_len * img_c)
num_actions = env.action_space.n

In [8]:
input_shape

(84, 84, 4)

In [9]:
# set up placeholders
# placeholder for current observation (or state)
obs_t_ph              = tf.placeholder(tf.uint8, [None] + list(input_shape))
# placeholder for current action
act_t_ph              = tf.placeholder(tf.int32,   [None])
# placeholder for current reward
rew_t_ph              = tf.placeholder(tf.float32, [None])
# placeholder for next observation (or state)
obs_tp1_ph            = tf.placeholder(tf.uint8, [None] + list(input_shape))
# placeholder for end of episode mask
# this value is 1 if the next state corresponds to the end of an episode,
# in which case there is no Q-value at the next state; at the end of an
# episode, only the current state reward contributes to the target, not the
# next state Q-value (i.e. target is just rew_t_ph, not rew_t_ph + gamma * q_tp1)
done_mask_ph          = tf.placeholder(tf.float32, [None])

# casting to float on GPU ensures lower data transfer times.
obs_t_float   = tf.cast(obs_t_ph,   tf.float32) / 255.0
obs_tp1_float = tf.cast(obs_tp1_ph, tf.float32) / 255.0

In [10]:
# Here, you should fill in your own code to compute the Bellman error. This requires
# evaluating the current and next Q-values and constructing the corresponding error.
# TensorFlow will differentiate this error for you, you just need to pass it to the
# optimizer. See assignment text for details.
# Your code should produce one scalar-valued tensor: total_error
# This will be passed to the optimizer in the provided code below.
# Your code should also produce two collections of variables:
# q_func_vars
# target_q_func_vars
# These should hold all of the variables of the Q-function network and target network,
# respectively. A convenient way to get these is to make use of TF's "scope" feature.
# For example, you can create your Q-function network with the scope "q_func" like this:
# <something> = q_func(obs_t_float, num_actions, scope="q_func", reuse=False)
# And then you can obtain the variables like this:
# q_func_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='q_func')
# Older versions of TensorFlow may require using "VARIABLES" instead of "GLOBAL_VARIABLES"
######

# YOUR CODE HERE
#get q value for all actions for the current step and for the next step
q_t_all = q_func(img_in=obs_t_float, num_actions=num_actions, scope="q_func", reuse=False)
greedy_action = tf.argmax(q_t_all, axis=1)

q_tp1_all = q_func(img_in=obs_tp1_float, num_actions=num_actions, scope="target_q_func", reuse=False)

q_func_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='q_func')
target_q_func_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_q_func')

q_t = tf.reduce_sum(tf.one_hot(act_t_ph, depth=num_actions, dtype=tf.float32) * q_t_all, axis=1)
v_tp1 = tf.reduce_max(q_tp1_all, axis=1)
q_target = rew_t_ph + (1.0 - done_mask_ph) * gamma * v_tp1

total_error = tf.losses.mean_squared_error(q_target, q_t)

In [11]:
######

# construct optimization op (with gradient clipping)
learning_rate = tf.placeholder(tf.float32, (), name="learning_rate")
optimizer = optimizer_spec.constructor(learning_rate=learning_rate, **optimizer_spec.kwargs)
train_fn = minimize_and_clip(optimizer, total_error,
             var_list=q_func_vars, clip_val=grad_norm_clipping)

# update_target_fn will be called periodically to copy Q network to target Q network
update_target_fn = []
for var, var_target in zip(sorted(q_func_vars,        key=lambda v: v.name),
                           sorted(target_q_func_vars, key=lambda v: v.name)):
    update_target_fn.append(var_target.assign(var))
update_target_fn = tf.group(*update_target_fn)

# construct the replay buffer
replay_buffer = ReplayBuffer(replay_buffer_size, frame_history_len)

###############
# RUN ENV     #
###############
model_initialized = False
num_param_updates = 0
mean_episode_reward      = -float('nan')
best_mean_episode_reward = -float('inf')
last_obs = env.reset()
LOG_EVERY_N_STEPS = 10000

[2018-06-21 01:31:55,616] Starting new video recorder writing to /tmp/hw3_vid_dir2/gym/openaigym.video.0.8907.video000000.mp4


In [ ]:
for t in itertools.count():
    ### 1. Check stopping criterion
    if stopping_criterion is not None and stopping_criterion(env, t):
        print("stopping")
        break

    ### 2. Step the env and store the transition
    # At this point, "last_obs" contains the latest observation that was
    # recorded from the simulator. Here, your code needs to store this
    # observation and its outcome (reward, next observation, etc.) into
    # the replay buffer while stepping the simulator forward one step.
    # At the end of this block of code, the simulator should have been
    # advanced one step, and the replay buffer should contain one more
    # transition.
    # Specifically, last_obs must point to the new latest observation.
    # Useful functions you'll need to call:
    # obs, reward, done, info = env.step(action)
    # this steps the environment forward one step
    # obs = env.reset()
    # this resets the environment if you reached an episode boundary.
    # Don't forget to call env.reset() to get a new observation if done
    # is true!!
    # Note that you cannot use "last_obs" directly as input
    # into your network, since it needs to be processed to include context
    # from previous frames. You should check out the replay buffer
    # implementation in dqn_utils.py to see what functionality the replay
    # buffer exposes. The replay buffer has a function called
    # encode_recent_observation that will take the latest observation
    # that you pushed into the buffer and compute the corresponding
    # input that should be given to a Q network by appending some
    # previous frames.
    # Don't forget to include epsilon greedy exploration!
    # And remember that the first time you enter this loop, the model
    # may not yet have been initialized (but of course, the first step
    # might as well be random, since you haven't trained your net...)

    #####

    # YOUR CODE HERE
    idx = replay_buffer.store_frame(last_obs)
    
    if not model_initialized or random.random() < exploration.value(t):
        action = random.randint(0, num_actions - 1)
    else:
        obs = replay_buffer.encode_recent_observation()
        action = session.run(greedy_action, {obs_t_ph: [obs]})
    
    next_obs, reward, done, _ = env.step(action)
    if done:
        last_obs = env.reset()

    replay_buffer.store_effect(idx, action, reward, done)
    last_obs = next_obs

    replay_buffer.store_effect(working_idx, action, reward, done)

    #####

    # at this point, the environment should have been advanced one step (and
    # reset if done was true), and last_obs should point to the new latest
    # observation

    ### 3. Perform experience replay and train the network.
    # note that this is only done if the replay buffer contains enough samples
    # for us to learn something useful -- until then, the model will not be
    # initialized and random actions should be taken
    if (t > learning_starts and
            t % learning_freq == 0 and
            replay_buffer.can_sample(batch_size)):
        # Here, you should perform training. Training consists of four steps:
        # 3.a: use the replay buffer to sample a batch of transitions (see the
        # replay buffer code for function definition, each batch that you sample
        # should consist of current observations, current actions, rewards,
        # next observations, and done indicator).
        # 3.b: initialize the model if it has not been initialized yet; to do
        # that, call
        #    initialize_interdependent_variables(session, tf.global_variables(), {
        #        obs_t_ph: obs_t_batch,
        #        obs_tp1_ph: obs_tp1_batch,
        #    })
        # where obs_t_batch and obs_tp1_batch are the batches of observations at
        # the current and next time step. The boolean variable model_initialized
        # indicates whether or not the model has been initialized.
        # Remember that you have to update the target network too (see 3.d)!
        # 3.c: train the model. To do this, you'll need to use the train_fn and
        # total_error ops that were created earlier: total_error is what you
        # created to compute the total Bellman error in a batch, and train_fn
        # will actually perform a gradient step and update the network parameters
        # to reduce total_error. When calling session.run on these you'll need to
        # populate the following placeholders:
        # obs_t_ph
        # act_t_ph
        # rew_t_ph
        # obs_tp1_ph
        # done_mask_ph
        # (this is needed for computing total_error)
        # learning_rate -- you can get this from optimizer_spec.lr_schedule.value(t)
        # (this is needed by the optimizer to choose the learning rate)
        # 3.d: periodically update the target network by calling
        # session.run(update_target_fn)
        # you should update every target_update_freq steps, and you may find the
        # variable num_param_updates useful for this (it was initialized to 0)
        #####

        # YOUR CODE HERE
        
        obs_t_batch, act_batch, rew_batch, obs_tp1_batch, done_mask = replay_buffer.sample(batch_size)
        
        
        
        session.run(train_fn, feed_dict={obs_t_ph: obs_t_batch, act_t_ph: act_batch, \
                    obs_tp1_ph: obs_tp1_batch, rew_t_ph: rew_batch, \
                    done_mask_ph: done_mask, learning_rate: optimizer_spec.lr_schedule.value(t)})
        
        if t and not  t%target_update_freq:
            session.run(update_target_fn)
            num_param_updates += 1
        #####
        

    ### 4. Log progress
    episode_rewards = get_wrapper_by_name(env, "Monitor").get_episode_rewards()
    if len(episode_rewards) > 0:
        mean_episode_reward = np.mean(episode_rewards[-100:])
    if len(episode_rewards) > 100:
        best_mean_episode_reward = max(best_mean_episode_reward, mean_episode_reward)
    if t % LOG_EVERY_N_STEPS == 0 and model_initialized:
        print("Timestep %d" % (t,))
        print("mean reward (100 episodes) %f" % mean_episode_reward)
        print("best mean reward %f" % best_mean_episode_reward)
        print("episodes %d" % len(episode_rewards))
        print("exploration %f" % exploration.value(t))
        print("learning_rate %f" % optimizer_spec.lr_schedule.value(t))
        sys.stdout.flush()

Timestep 0
mean reward (100 episodes) -19.760000
best mean reward -19.720000
episodes 380
exploration 1.000000
learning_rate 0.000100
Timestep 10000
mean reward (100 episodes) -19.780000
best mean reward -19.690000
episodes 390
exploration 0.991000
learning_rate 0.000100
Timestep 20000
mean reward (100 episodes) -19.770000
best mean reward -19.690000
episodes 400
exploration 0.982000
learning_rate 0.000100
Timestep 30000
mean reward (100 episodes) -19.840000
best mean reward -19.690000
episodes 411
exploration 0.973000
learning_rate 0.000097
Timestep 40000
mean reward (100 episodes) -19.880000
best mean reward -19.690000
episodes 422
exploration 0.964000
learning_rate 0.000092
Timestep 50000
mean reward (100 episodes) -19.910000
best mean reward -19.690000
episodes 433
exploration 0.955000
learning_rate 0.000087
Timestep 60000
mean reward (100 episodes) -19.960000
best mean reward -19.690000
episodes 443
exploration 0.946000
learning_rate 0.000082
Timestep 70000
mean reward (100 episod

[2018-06-21 02:33:59,576] Starting new video recorder writing to /tmp/hw3_vid_dir2/gym/openaigym.video.0.8907.video000512.mp4


Timestep 130000
mean reward (100 episodes) -19.880000
best mean reward -19.690000
episodes 512
exploration 0.883000
learning_rate 0.000050
Timestep 140000
mean reward (100 episodes) -19.820000
best mean reward -19.690000
episodes 521
exploration 0.874000
learning_rate 0.000050
Timestep 150000
mean reward (100 episodes) -19.780000
best mean reward -19.690000
episodes 531
exploration 0.865000
learning_rate 0.000050
Timestep 160000
mean reward (100 episodes) -19.760000
best mean reward -19.690000
episodes 541
exploration 0.856000
learning_rate 0.000050
Timestep 170000
mean reward (100 episodes) -19.690000
best mean reward -19.680000
episodes 550
exploration 0.847000
learning_rate 0.000050
Timestep 180000
mean reward (100 episodes) -19.670000
best mean reward -19.620000
episodes 560
exploration 0.838000
learning_rate 0.000050
Timestep 190000
mean reward (100 episodes) -19.610000
best mean reward -19.580000
episodes 568
exploration 0.829000
learning_rate 0.000050
Timestep 200000
mean reward

[2018-06-21 02:46:26,232] Starting new video recorder writing to /tmp/hw3_vid_dir2/gym/openaigym.video.0.8907.video000729.mp4


Timestep 390000
mean reward (100 episodes) -18.660000
best mean reward -18.640000
episodes 731
exploration 0.649000
learning_rate 0.000050
Timestep 400000
mean reward (100 episodes) -18.540000
best mean reward -18.540000
episodes 738
exploration 0.640000
learning_rate 0.000050
Timestep 410000
mean reward (100 episodes) -18.510000
best mean reward -18.490000
episodes 745
exploration 0.631000
learning_rate 0.000050
Timestep 420000
mean reward (100 episodes) -18.390000
best mean reward -18.390000
episodes 751
exploration 0.622000
learning_rate 0.000050
Timestep 430000
mean reward (100 episodes) -18.360000
best mean reward -18.360000
episodes 758
exploration 0.613000
learning_rate 0.000050
Timestep 440000
mean reward (100 episodes) -18.270000
best mean reward -18.250000
episodes 765
exploration 0.604000
learning_rate 0.000050
Timestep 450000
mean reward (100 episodes) -18.160000
best mean reward -18.160000
episodes 772
exploration 0.595000
learning_rate 0.000050
Timestep 460000
mean reward

...

Timestep 48600000
mean reward (100 episodes) 17.190000
best mean reward 18.810000
episodes 25616
exploration 0.100000
learning_rate 0.000050
Timestep 48610000
mean reward (100 episodes) 17.180000
best mean reward 18.810000
episodes 25622
exploration 0.100000
learning_rate 0.000050
Timestep 48620000
mean reward (100 episodes) 17.130000
best mean reward 18.810000
episodes 25627
exploration 0.100000
learning_rate 0.000050
Timestep 48630000
mean reward (100 episodes) 17.190000
best mean reward 18.810000
episodes 25632
exploration 0.100000
learning_rate 0.000050
Timestep 48640000
mean reward (100 episodes) 17.230000
best mean reward 18.810000
episodes 25637
exploration 0.100000
learning_rate 0.000050
Timestep 48650000
mean reward (100 episodes) 17.290000
best mean reward 18.810000
episodes 25643
exploration 0.100000
learning_rate 0.000050
Timestep 48660000
mean reward (100 episodes) 17.260000
best mean reward 18.810000
episodes 25648
exploration 0.100000
learning_rate 0.000050
Timestep 4867

Timestep 49190000
mean reward (100 episodes) 17.650000
best mean reward 18.810000
episodes 25920
exploration 0.100000
learning_rate 0.000050
Timestep 49200000
mean reward (100 episodes) 17.690000
best mean reward 18.810000
episodes 25926
exploration 0.100000
learning_rate 0.000050
Timestep 49210000
mean reward (100 episodes) 17.670000
best mean reward 18.810000
episodes 25931
exploration 0.100000
learning_rate 0.000050
Timestep 49220000
mean reward (100 episodes) 17.680000
best mean reward 18.810000
episodes 25936
exploration 0.100000
learning_rate 0.000050
Timestep 49230000
mean reward (100 episodes) 17.680000
best mean reward 18.810000
episodes 25942
exploration 0.100000
learning_rate 0.000050
Timestep 49240000
mean reward (100 episodes) 17.650000
best mean reward 18.810000
episodes 25947
exploration 0.100000
learning_rate 0.000050
Timestep 49250000
mean reward (100 episodes) 17.640000
best mean reward 18.810000
episodes 25952
exploration 0.100000
learning_rate 0.000050
Timestep 4926

[2018-06-23 00:46:07,845] Starting new video recorder writing to /tmp/hw3_vid_dir2/gym/openaigym.video.0.8907.video026000.mp4


Timestep 49350000
mean reward (100 episodes) 17.630000
best mean reward 18.810000
episodes 26004
exploration 0.100000
learning_rate 0.000050
Timestep 49360000
mean reward (100 episodes) 17.650000
best mean reward 18.810000
episodes 26009
exploration 0.100000
learning_rate 0.000050
Timestep 49370000
mean reward (100 episodes) 17.790000
best mean reward 18.810000
episodes 26015
exploration 0.100000
learning_rate 0.000050
Timestep 49380000
mean reward (100 episodes) 17.810000
best mean reward 18.810000
episodes 26020
exploration 0.100000
learning_rate 0.000050
Timestep 49390000
mean reward (100 episodes) 17.730000
best mean reward 18.810000
episodes 26025
exploration 0.100000
learning_rate 0.000050
Timestep 49400000
mean reward (100 episodes) 17.610000
best mean reward 18.810000
episodes 26030
exploration 0.100000
learning_rate 0.000050
Timestep 49410000
mean reward (100 episodes) 17.600000
best mean reward 18.810000
episodes 26035
exploration 0.100000
learning_rate 0.000050
Timestep 4942

Timestep 49940000
mean reward (100 episodes) 17.630000
best mean reward 18.810000
episodes 26315
exploration 0.100000
learning_rate 0.000050
Timestep 49950000
mean reward (100 episodes) 17.610000
best mean reward 18.810000
episodes 26320
exploration 0.100000
learning_rate 0.000050
Timestep 49960000
mean reward (100 episodes) 17.710000
best mean reward 18.810000
episodes 26326
exploration 0.100000
learning_rate 0.000050
Timestep 49970000
mean reward (100 episodes) 17.790000
best mean reward 18.810000
episodes 26331
exploration 0.100000
learning_rate 0.000050
Timestep 49980000
mean reward (100 episodes) 17.670000
best mean reward 18.810000
episodes 26336
exploration 0.100000
learning_rate 0.000050
Timestep 49990000
mean reward (100 episodes) 17.680000
best mean reward 18.810000
episodes 26342
exploration 0.100000
learning_rate 0.000050
Timestep 50000000
mean reward (100 episodes) 17.680000
best mean reward 18.810000
episodes 26347
exploration 0.100000
learning_rate 0.000050
Timestep 5001

In [ ]:
replay_buffer = ReplayBuffer(1000, 10)

In [41]:
isinstance(env, gym.Wrapper)

False